In [1]:
#Dependencies
import gmaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from scipy.stats import linregress
from math import *
import datetime

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
# The path to our bus stop locations CSV file
locations_file = "Resources/stops.csv"

# Read our file data into pandas
locations_df = pd.read_csv(locations_file)
locations_df.head()

NameError: name 'pd' is not defined

In [3]:
# The path to our bus stop times CSV file
stops_file = "Resources/stop_times.csv"

# Read our file data into pandas
stops_df = pd.read_csv(stops_file)
stops_df.head()

NameError: name 'pd' is not defined

In [4]:
# merge
bus_data = pd.merge(locations_df, stops_df, on=["stop_id", "stop_id"])
bus_data.head()

NameError: name 'pd' is not defined

In [5]:
#groupby stop id to see how many stops 
grouped_bystops = bus_data.groupby(['stop_id', 'stop_lat', 'stop_lon'])
numberstops = grouped_bystops['arrival_time'].nunique()
numberstops

NameError: name 'bus_data' is not defined

In [6]:
#numberstops.tolist()

In [7]:
stopnum_df = pd.DataFrame({"Number of Stops":numberstops})
stopnum_df

NameError: name 'pd' is not defined

In [8]:
# merge stops with lat and long
stopnum_df.reset_index(inplace = True)


NameError: name 'stopnum_df' is not defined

In [9]:
# Store lat and long
locations = stopnum_df[['stop_lat','stop_lon']]
stops = stopnum_df[['Number of Stops']].astype(float)

NameError: name 'stopnum_df' is not defined

In [10]:
# Plot Heatmap
fig_heat = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=2, point_radius=.0025)
fig_heat.add_layer(heat_layer)
fig_heat

NameError: name 'gmaps' is not defined

In [11]:
# plot locations
fig_mark = gmaps.figure()
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig_mark.add_layer(markers)
fig_mark

NameError: name 'gmaps' is not defined

In [12]:
# Add destination coordinates for Public Square
dest_coords = (41.4997,81.6937)

#stopnum_df["Distance (m)"] = ""

In [13]:
#Drop rows with bad data
clean_trip = stops_df.copy()
clean_trip.dropna(how="any",inplace=True)
clean_trip

NameError: name 'stops_df' is not defined

In [14]:
clean_trip.set_index("arrival_time")
clean_trip.head()

NameError: name 'clean_trip' is not defined

In [15]:
#Convert arrival and departure times into timestamps
clean_trip["arrival_time"] = pd.to_datetime(clean_trip["arrival_time"])
clean_trip["departure_time"] = pd.to_datetime(clean_trip["departure_time"])
clean_trip

NameError: name 'pd' is not defined

In [16]:
trip_group = clean_trip.groupby("trip_id").count()
trip_group["min_arr"] = ""
trip_group["max_arr"] = ""
trip_group

NameError: name 'clean_trip' is not defined

In [17]:
#Make sure that trips are one way
clean_trip.loc[clean_trip["trip_id"] == 16723936]

NameError: name 'clean_trip' is not defined

In [18]:
trip_group.reset_index(inplace=True)
#trip_group.head()

min_trips = clean_trip.groupby(["trip_id"]).agg({"arrival_time":["min","max"]})
min_trips

NameError: name 'trip_group' is not defined

In [19]:
min_trips["duration"] = min_trips["arrival_time"]["max"] - min_trips["arrival_time"]["min"]

min_trips

NameError: name 'min_trips' is not defined

In [20]:
#Get longest trips
longest_trip = min_trips["duration"].max()

min_trips.loc[min_trips["duration"] == longest_trip]

NameError: name 'min_trips' is not defined

In [21]:
bus_data.head()

NameError: name 'bus_data' is not defined

In [22]:
lngtripmap = bus_data.loc[bus_data["trip_id"] == 16741726]

lngtripmap = lngtripmap[["stop_id","stop_name","stop_lat","stop_lon","trip_id","arrival_time","departure_time","stop_sequence"]]

NameError: name 'bus_data' is not defined

In [23]:
lngtripmap.sort_values(by="stop_sequence")

NameError: name 'lngtripmap' is not defined

In [24]:
#Diplay route with longest commute time
lngrte_fig = gmaps.figure()

lngtripmap.reset_index(inplace=True)
lngrte = lngtripmap[["stop_lat","stop_lon"]]
lngrte_stop_name = lngtripmap["stop_name"]
lngrte_arr_time = lngtripmap["arrival_time"]

# info_box_template = """
# <dl>
# <dt>Arrival Time</dt><dd>{lngrte_arr_time}</dd>
# </dl>
# """

# times = [info_box_template.format(**lngrte_arr_time) for time in lngrte_arr_time]


lngrte_layer = gmaps.marker_layer(lngrte,hover_text=lngrte_stop_name,info_box_content=lngrte_arr_time)
lngrte_fig.add_layer(lngrte_layer)
lngrte_fig

NameError: name 'gmaps' is not defined

In [25]:
#Get trip distances
distance_df = bus_data[["stop_id","stop_name","stop_lat","stop_lon","trip_id","stop_sequence"]]
distance_df.head()


NameError: name 'bus_data' is not defined

In [26]:
#Get all last stops by trip
last_stop = distance_df.groupby(["trip_id"],as_index=False).agg({"stop_sequence":["max"]})

last_stop.columns = last_stop.columns.get_level_values(0)

stop_dist_df = pd.merge(last_stop,distance_df,on=["trip_id","stop_sequence"])
stop_dist_df


NameError: name 'distance_df' is not defined

In [27]:
#Get all first stops by trip
first_stop = distance_df.groupby(["trip_id"],as_index=False).agg({"stop_sequence":["min"]})

first_stop.columns = first_stop.columns.get_level_values(0)

start_dist_df = pd.merge(first_stop,distance_df,on=["trip_id","stop_sequence"])
start_dist_df

NameError: name 'distance_df' is not defined

In [28]:
#Get distances for start and end stops using Haversine formula found on Stack Overflow
def haversine_np(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    mi = 3956 * c
    return mi

In [29]:
#Put distances for each trip into new column
start_dist_df["Distance (mi)"] = haversine_np(start_dist_df["stop_lon"],start_dist_df["stop_lat"],stop_dist_df["stop_lon"],stop_dist_df["stop_lat"])
start_dist_df

NameError: name 'start_dist_df' is not defined

In [30]:
min_trips.columns = min_trips.columns.get_level_values(0)
commute_data = pd.merge(start_dist_df,min_trips,on="trip_id")
commute_data

NameError: name 'min_trips' is not defined

In [31]:
commute_data["Duration (minutes)"] = commute_data["duration"].dt.seconds / 60
commute_data

NameError: name 'commute_data' is not defined

In [32]:
#Plot distance versus duration of trip
x_axis = commute_data["Duration (minutes)"].astype(float)
y_axis = commute_data["Distance (mi)"].astype(float)

plt.scatter(x_axis,y_axis,color="b",edgecolors="black")

(slope,intercept,rvalue,pvalue,stderr) = linregress(x_axis,y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.plot(x_axis,regress_values,"-r")
plt.xlabel("Duration (minutes)")
plt.ylabel("Distance (miles)")
plt.title("Duration of RTA Trips by Distance")
plt.annotate(line_eq,(60,4.5),fontsize=15,color="red")

plt.show()
plt.savefig("Results/commute_times.png")

print(f"The r-squared value is {rvalue**2}.")

NameError: name 'commute_data' is not defined

In [33]:
x_axis = commute_data["Duration (minutes)"].astype(float)
y_axis = trip_group["stop_sequence"]

plt.scatter(x_axis,y_axis,color="b",edgecolors="black")

(slope,intercept,rvalue,pvalue,stderr) = linregress(x_axis,y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

plt.plot(x_axis,regress_values,"-r")
plt.xlabel("Duration (minutes)")
plt.ylabel("Number of Stops")
plt.title("Duration of RTA Trips by Number of Stops")
plt.annotate(line_eq,(60,4.5),fontsize=15,color="red")

plt.show()
plt.savefig("Results/commute_numstops.png")

print(f"The r-squared value is {rvalue**2}.")

NameError: name 'commute_data' is not defined